# Neo4j Import

Now that we have our scraped tweet data we're ready to insert into Neo4j. We'll do this by loading the JSON data and passing it as a parameter to a Cypher query, using the [official Python driver for Neo4j](https://github.com/neo4j/neo4j-python-driver).

*Note: don't forget to start the Neo4j service*: `sudo service neo4j start`

In [2]:
import os
import json
from dotenv import find_dotenv, load_dotenv
from neo4j.v1 import GraphDatabase  # official Neo4j driver for Python

In [8]:
load_dotenv(find_dotenv(".env"))

True

In [9]:
NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")

In [11]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [15]:
with open('./data/tweets_full.json') as json_data:
    valid_tweets = json.load(json_data)

In [16]:
len(valid_tweets)

1463

## Delete everything in Neo4j

In [20]:
delete_query = '''
MATCH (n)
DETACH DELETE (n)
'''

In [21]:
%%time
with driver.session() as session:
    session.run(delete_query)

CPU times: user 2.68 ms, sys: 0 ns, total: 2.68 ms
Wall time: 341 ms


## Check that Neo4j contains no data

In [40]:
%%time
with driver.session() as session:
    res = session.run("MATCH (n) RETURN (n)")

CPU times: user 1.54 ms, sys: 189 µs, total: 1.73 ms
Wall time: 2.33 ms


In [42]:
res.data()

[]

## Import all valid tweets into Neo4j

In [43]:
import_query = '''
WITH $valid_tweets AS tweets
UNWIND tweets AS tweet
MERGE (u:User {user_id: tweet.user_id})
ON CREATE SET u.screen_name = tweet.screen_name
MERGE (t:Tweet {tweet_id: tweet.tweet_id})
ON CREATE SET
  t.text = tweet.tweet_text,
  t.permalink = tweet.permalink
MERGE (u)-[:POSTED]->(t)

FOREACH (ht IN tweet.hashtags |
  MERGE (h:Hashtag {tag: ht.tag })
  ON CREATE SET h.archived_url = ht.archived_url
  MERGE (t)-[:HAS_TAG]->(h)
)

FOREACH (link IN tweet.links |
  MERGE (l:Link {url: link.url})
  ON CREATE SET l.archived_url = link.archived_url
  MERGE (t)-[:HAS_LINK]->(l)
)
'''

It's better to use a transaction to import the data.

In [45]:
%%time
def add_tweets(tx):
    tx.run(import_query, valid_tweets=valid_tweets)

with driver.session() as session:
    session.write_transaction(unit_of_work=add_tweets)

CPU times: user 149 ms, sys: 0 ns, total: 149 ms
Wall time: 4.91 s


## Check that tweets were stored in Neo4j

We can check that the data was succesfully inserted by running a simple query.

In [66]:
query = '''
MATCH (u:User)-[:POSTED]->(t:Tweet)
MATCH (t)-[:HAS_TAG]->(h:Hashtag)
WHERE h.tag = 'superbowlfood'
RETURN u, t, h
'''

In [67]:
%%time
with driver.session() as session:
    res = session.run(query)

CPU times: user 3.6 ms, sys: 0 ns, total: 3.6 ms
Wall time: 14.4 ms


In [68]:
res.statement

"\nMATCH (u:User)-[:POSTED]->(t:Tweet)\nMATCH (t)-[:HAS_TAG]->(h:Hashtag)\nWHERE h.tag = 'superbowlfood'\nRETURN u, t, h\n"

In [69]:
res.data()

[{'u': <Node id=6212 labels={'User'} properties={'screen_name': 'AlwaysHungryBae', 'user_id': '2882130846'}>,
  't': <Node id=6537 labels={'Tweet'} properties={'permalink': '/AlwaysHungryBae/status/561931644785811457', 'tweet_id': '561931644785811457', 'text': 'Happy Super Bowl Sunday \n'}>,
  'h': <Node id=7989 labels={'Hashtag'} properties={'tag': 'superbowlfood', 'archived_url': '/web/20150603004258/https://twitter.com/hashtag/superbowlfood?src=hash'}>},
 {'u': <Node id=6213 labels={'User'} properties={'user_id': '61483830', 'screen_name': 'BMoreBirdsNest'}>,
  't': <Node id=6538 labels={'Tweet'} properties={'permalink': '/BMoreBirdsNest/status/561917739108155392', 'tweet_id': '561917739108155392', 'text': 'Making the award-winning Pigs in a Pillow again today! Bacon, goat cheese, jam, glazed donut as bread. '}>,
  'h': <Node id=7989 labels={'Hashtag'} properties={'tag': 'superbowlfood', 'archived_url': '/web/20150603004258/https://twitter.com/hashtag/superbowlfood?src=hash'}>},
 {'

!["Graph of users who tweetes #superbowlfood"](../img/superbowlfood.png)

In [89]:
query = '''
// What are the 10 most common hashtags
MATCH (u:User)-[:POSTED]->(t:Tweet)-[:HAS_TAG]->(h:Hashtag)
RETURN h.tag AS hashtag, COUNT(*) AS num
ORDER BY num DESC
LIMIT 10
'''

In [91]:
with driver.session() as session:
    res = session.run(query)
res.data()

[{'hashtag': 'politics', 'num': 57},
 {'hashtag': 'news', 'num': 30},
 {'hashtag': 'crime', 'num': 21},
 {'hashtag': 'sports', 'num': 19},
 {'hashtag': 'Cleveland', 'num': 17},
 {'hashtag': 'TrumpForPresident', 'num': 16},
 {'hashtag': 'tcot', 'num': 12},
 {'hashtag': 'ColumbianChemicals', 'num': 11},
 {'hashtag': 'MAGA', 'num': 11},
 {'hashtag': 'Chicago', 'num': 11}]

In [92]:
query = '''
// What hashtags are used together most frequently
MATCH (h1:Hashtag)<-[:HAS_TAG]-(t:Tweet)-[:HAS_TAG]->(h2:Hashtag)
WHERE id(h1) < id(h2)
RETURN h1.tag, h2.tag, COUNT(*) AS num
ORDER BY num DESC
LIMIT 5
'''

In [93]:
with driver.session() as session:
    res = session.run(query)
res.data()

[{'h1.tag': 'sports', 'h2.tag': 'Cleveland', 'num': 8},
 {'h1.tag': 'TrumpForPresident', 'h2.tag': 'HillaryForPrison2016', 'num': 6},
 {'h1.tag': 'politics', 'h2.tag': 'Cleveland', 'num': 6},
 {'h1.tag': 'ColumbianChemicals', 'h2.tag': 'DeadHorse', 'num': 5},
 {'h1.tag': 'TrumpRally', 'h2.tag': 'Melbourne', 'num': 4}]

In [100]:
query = '''
// Most common domains shared in tweets
MATCH (t:Tweet)-[:HAS_LINK]->(u:Link)
WITH t, replace(replace(u.url, "http://", '' ), "https://", '') AS url
RETURN COUNT(t) AS num, head(split(url, "/")) AS domain
ORDER BY num
DESC LIMIT 10
'''

In [101]:
with driver.session() as session:
    res = session.run(query)
res.data()

[{'num': 836, 'domain': 'pic.twitter.com'},
 {'num': 91, 'domain': 'bit.ly'},
 {'num': 80, 'domain': 'pbs.twimg.com'},
 {'num': 62, 'domain': 'twitter.com'},
 {'num': 21, 'domain': 'riafan.ru'},
 {'num': 20, 'domain': 'inforeactor.ru'},
 {'num': 17, 'domain': 'goodspb.livejournal.com'},
 {'num': 15, 'domain': 'www.fox5atlanta.com'},
 {'num': 14, 'domain': 'goo.gl'},
 {'num': 6, 'domain': 'blackmattersus.com'}]

## Define GraphQL schema

In [70]:
graphQL_schema = '''

type Tweet {
    tweet_id: ID!
    text: String
    permalink: String
    author: User @relation(name: "POSTED", direction: "IN")
    hashtags: [Hashtag] @relation(name: "HAS_TAG", direction: "IN")
    links: [Link] @relation(name: "HAS_LINK", direction: "IN")
}

type User {
    user_id: ID!
    screen_name: String
    tweets: [Tweet] @relation(name: "POSTED", direction: "OUT")
}

type Hashtag {
    tag: ID!
    archived_url: String
    tweets: [Tweet] @relation(name: "HAS_TAG", direction: "IN")
}

type Link {
    url: ID!
    archived_url: String
}
'''